In [10]:
import pandas as pd
import numpy as np
import datetime
import pickle 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, roc_auc_score
import seaborn as sns
from bioinfokit.analys import stat, get_data
from sklearn.metrics import f1_score
from sklearn import metrics
import sklearn.metrics as metrics
from scipy.stats import chisquare
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [2]:
df = pd.read_csv('/Users/markdetiberiis/Desktop/Patient_no_show/model_ready_data.csv')

In [3]:
df.drop(['Unnamed: 0'],axis = 1,inplace=True)

In [4]:
df

,Gender,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSReceived,NoShow,appointment_weekday,schedule_weekday,waiting_days,schedule_date_Y,schedule_date_M,schedule_date_D,appointment_date_Y,appointment_date_M,appointment_date_D
0,0,62,39,0,1,0,0,0,0,0,0,0,2,2016,4,29,2016,4,29
1,1,56,39,0,0,0,0,0,0,0,0,0,2,2016,4,29,2016,4,29
2,0,62,45,0,0,0,0,0,0,0,0,0,2,2016,4,29,2016,4,29
3,0,8,54,0,0,0,0,0,0,0,0,0,2,2016,4,29,2016,4,29
4,0,56,39,0,1,1,0,0,0,0,0,0,2,2016,4,29,2016,4,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,0,56,43,0,0,0,0,0,1,0,4,4,61,2016,5,3,2016,6,7
110523,0,51,43,0,0,0,0,0,1,0,4,4,61,2016,5,3,2016,6,7
110524,0,21,43,0,0,0,0,0,1,0,4,5,68,2016,4,27,2016,6,7
110525,0,38,43,0,0,0,0,0,1,0,4,5,68,2016,4,27,2016,6,7


In [9]:
df.corr(method = 'spearman')

,Gender,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSReceived,NoShow,appointment_weekday,schedule_weekday,waiting_days,schedule_date_Y,schedule_date_M,schedule_date_D,appointment_date_Y,appointment_date_M,appointment_date_D
Gender,1.000000,-0.107099,-0.007776,-0.114293,-0.055718,-0.032554,0.106167,0.022437,-0.046298,-0.004119,-0.007309,-0.012567,-0.043711,0.003766,0.015522,0.003380,NaN,-0.006001,-0.000232
Age,-0.107099,1.000000,-0.015293,-0.090099,0.503204,0.293320,0.102266,0.078889,0.015389,-0.061228,0.002380,-0.016175,0.075073,-0.025872,0.001867,-0.011060,NaN,0.014434,-0.009868
Neighbourhood,-0.007776,-0.015293,1.000000,0.001659,0.005844,0.012162,-0.000484,0.011010,0.017515,-0.009515,0.004393,0.011846,-0.007170,-0.018328,0.005038,0.003486,NaN,0.002611,0.001355
Scholarship,-0.114293,-0.090099,0.001659,1.000000,-0.019729,-0.024894,0.035022,-0.009094,0.001194,0.029135,0.000366,0.009018,-0.023247,0.001402,0.013161,0.006932,NaN,-0.002642,0.001553
Hypertension,-0.055718,0.503204,0.005844,-0.019729,1.000000,0.433086,0.087971,0.084845,-0.006267,-0.035701,0.000278,-0.004505,0.027022,-0.035310,0.022451,0.000371,NaN,0.003676,-0.009519
Diabetes,-0.032554,0.293320,0.012162,-0.024894,0.433086,1.000000,0.018474,0.059180,-0.014550,-0.015180,-0.004146,-0.002816,0.005954,-0.006724,0.027473,-0.007231,NaN,0.004004,-0.004728
Alcoholism,0.106167,0.102266,-0.000484,0.035022,0.087971,0.018474,1.000000,0.003702,-0.026147,-0.000196,-0.003736,-0.006980,-0.034050,0.004195,0.029916,-0.004224,NaN,0.004164,-0.003106
Handicap,0.022437,0.078889,0.011010,-0.009094,0.084845,0.059180,0.003702,1.000000,-0.025221,-0.007265,-0.003748,-0.002255,-0.022004,-0.007417,0.017358,-0.000852,NaN,-0.001529,-0.001719
SMSReceived,-0.046298,0.015389,0.017515,0.001194,-0.006267,-0.014550,-0.026147,-0.025221,1.000000,0.126431,0.095229,-0.028828,0.534123,-0.018903,-0.317134,0.165175,NaN,0.113767,-0.105541
NoShow,-0.004119,-0.061228,-0.009515,0.029135,-0.035701,-0.015180,-0.000196,-0.007265,0.126431,1.000000,-0.012135,-0.001088,0.236884,-0.006168,-0.172843,0.047467,NaN,-0.021785,0.008339


In [17]:
import researchpy as rp
import scipy.stats as stats

In [19]:
#ind t test

rp.ttest(group1= df['NoShow'][df['Gender'] == 0], group1_name= "Male",
         group2= df['NoShow'][df['Gender'] == 1], group2_name= "Female")

(   Variable         N      Mean        SD        SE  95% Conf.  Interval
 0      Male   71840.0  0.203146  0.402343  0.001501   0.200204  0.206088
 1    Female   38687.0  0.199679  0.399765  0.002032   0.195696  0.203663
 2  combined  110527.0  0.201933  0.401444  0.001208   0.199566  0.204299,
               Independent t-test      results
 0  Difference (Male - Female) =        0.0035
 1          Degrees of freedom =   110525.0000
 2                           t =        1.3693
 3       Two side test p value =        0.1709
 4      Difference < 0 p value =        0.9145
 5      Difference > 0 p value =        0.0855
 6                   Cohen's d =        0.0086
 7                   Hedge's g =        0.0086
 8               Glass's delta =        0.0086
 9                           r =        0.0041)

In [25]:
df.corr()['NoShow']

Gender                -0.004119
Age                   -0.060319
Neighbourhood         -0.008962
Scholarship            0.029135
Hypertension          -0.035701
Diabetes              -0.015180
Alcoholism            -0.000196
Handicap              -0.006076
SMSReceived            0.126431
NoShow                 1.000000
appointment_weekday   -0.013070
schedule_weekday      -0.000378
waiting_days           0.207676
schedule_date_Y       -0.006168
schedule_date_M       -0.160862
schedule_date_D        0.046557
appointment_date_Y          NaN
appointment_date_M    -0.020886
appointment_date_D     0.003763
Name: NoShow, dtype: float64

In [29]:
#chi sq test 

res = stat()
res.chisq(df=df)